In [9]:
import torch
import torch.nn as nn

class CRF(nn.Module):
    def __init__(
        self, 
        num_tags: int,
        batch_first: bool = True
    ):
        """init parameters of CRF
        
        Args:
            num_tags(int): number of tags
            batch_first(bool): 
        """
        super(CRF, self).__init__()
        self.num_tags = num_tags
        self.batch_first = batch_first
        
        self.transitions = nn.Parameter(torch.Tensor(num_tags, num_tags))
        self.start_transitions = nn.Parameter(torch.Tensor(num_tags))
        self.end_transitions = nn.Parameter(torch.Tensor(num_tags))
        
    def forward(
        self,
        emissions: torch.Tensor,
        tags: torch.LongTensor,
        mask: torch.ByteTensor | None = None
    ) -> torch.Tensor:
        """计算给定标签序列的负对数似然

        Args:
            emissions (torch.Tensor): _description_
            tags (torch.LongTensor): _description_
            mask (torch.ByteTensor | None, optional): _description_. Defaults to None.
        
        Returns:
            torch.tensor: 输入tags的负对数似然
        """
        if mask is None:
            mask = torch.ones_like(tags)
            
        if self.batch_fisrt:
            emissions = emissions.permute(1, 0, 2)
            tags = tags.permute(1, 0)
            mask = mask.permute(1, 0)
            
        score = self._compute_score(emissions, tags, mask)
        partition = self._compute_partition(emissions, mask)
        
        return partition - score
    
    def _compute_score(
        self,
        emissions: torch.Tensor,
        tags: torch.LongTensor,
        mask: torch.ByteTensor
    ) -> torch.Tensor:
        """_summary_

        Args:
            emissions (torch.Tensor): _description_
            tags (torch.LongTensor): _description_
            mask (torch.ByteTensor): _description_

        Returns:
            torch.Tensor: _description_
        """
        seq_len, batch_size = tags.shape
        first_tags = tags[0]
        
        score = self.start_transitions[first_tags]
        score += emissions[0, torch.arange(batch_size), first_tags]
        
        mask = mask.type_as(emissions)
        for i in range(1, seq_len):
            score += (
                self.transitions[tags[i], tags[i + 1]] + 
                emissions[i, torch.arange(batch_size), tags[i]]
            ) * mask[i]
            
        last_valid_index = mask.long().sum(dim=0) - 1
        last_tags = tags[last_valid_index, torch.arange(batch_size)]
        score += self.end_transitions[last_tags]
        
        return score
    
    def _compute_partition(
        self,
        emissions: torch.Tensor,
        mask: torch.ByteTensor
    ) -> torch.Tensor:
        """_summary_

        Args:
            emissions (torch.Tensor): _description_
            mask (torch.ByteTensor): _description_

        Returns:
            torch.Tensor: _description_
        """
        
        seq_len = emissions.shape[0]
        score = self.start_transitions.unsqueeze(0) + emissions[0]
        
        for i in range(1, seq_len):
            broadcast_score = score.unsqueeze(2)
            broadcast_emissions = emissions[i].unsqueeze(1)
            current_score = broadcast_score + self.transitions + broadcast_emissions
            current_score = torch.logsumexp(current_score, dim=1) # shape (batch_size, num_tags)
            score = torch.where(mask[i].bool().unsqueeze(1), current_score, score)
            
        score += self.end_transitions
        return torch.logsumexp(score, dim=1)

In [ ]:
# score + self.transitions + self.emissions[i]

tensor([[[-3.0241,  1.2105, -0.4853, -1.8851],
         [ 1.3328,  2.4362,  0.1036, -1.6654],
         [ 0.0840, -1.7990, -1.9624, -1.1515],
         [ 1.4715,  0.1385, -2.2108,  0.3224]],

        [[-0.5107,  0.7014, -0.0864, -1.8574],
         [-0.1873,  0.8551,  1.8883, -1.1910],
         [-0.4224, -2.7012, -0.5322,  2.8952],
         [-1.6950,  0.2984, -0.5764,  0.5523]],

        [[-1.2533,  0.6224, -2.0890, -1.1811],
         [-0.6233,  1.5568,  1.8633, -3.8581],
         [ 2.8365, -2.3353, -2.9130,  1.2665],
         [ 1.1481, -0.3084, -1.2882,  2.1339]]])